In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import nltk
import re
import numpy as np 
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer, PorterStemmer
from wordcloud import WordCloud, STOPWORDS

df = pd.read_csv(r"C:\Users\jim13\Documents\project1\youngadults2016-2021.csv")
df

,Unnamed: 0,author,created_utc,domain,full_link,num_comments,over_18,score,selftext,subreddit,title,num_crossposts
0,0,wise08,1500489233,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,0,False,1,We are hoping to improve services for young ad...,youngadults,Have you experienced abuse in a relationship?,NaN
1,1,[deleted],1500461003,culturewhisper.com,https://www.reddit.com/r/youngadults/comments/...,0,False,1,[deleted],youngadults,Best Young Adult books of the year?,NaN
2,2,dotandtittle,1499487580,i.redd.it,https://www.reddit.com/r/youngadults/comments/...,0,False,1,NaN,youngadults,I would love some feedback on my story The Gir...,NaN
3,3,[deleted],1499486812,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,0,False,1,[deleted],youngadults,I would love some feedback on my story The Gir...,NaN
4,4,[deleted],1499331661,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,3,False,3,[deleted],youngadults,why tf yall up,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6025,6025,pygmypuffonacid,1603368403,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,0,False,1,Do not ask me what we had to do to acquire the...,youngadults,And Now we have a 20 foot tall inflatable gori...,0.0
6026,6026,[deleted],1603366468,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,0,False,1,NaN,youngadults,What is your favorite realistic film like Offi...,0.0
6027,6027,MooseAdorable,1603349492,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,1,False,1,Hi there! \n\nMy name is Rose. I’m 22 years ol...,youngadults,Advice about a Job/Life,0.0
6028,6028,muzza1319,1603345177,self.youngadults,https://www.reddit.com/r/youngadults/comments/...,1,False,1,22 M from Australia we are in a strict lockdow...,youngadults,Anyone up for a conversation?,0.0


In [2]:
#Take interesting columns
df1=df[['selftext','num_comments','score','title']]
#drop those with text=NA
df1=df1.dropna(subset=['selftext'])
#drop meaningless text
df1=df1[(df1.selftext!='[removed]') & (df1.selftext!='[deleted]')].reset_index()
#drop duplicate
duplicate = df1.duplicated()
print('There are', duplicate.sum(), 'duplicated values.')
df1=df1.drop_duplicates()
len(df1)

There are 0 duplicated values.


4213

In [3]:
#convert to lower case
df1['selftext']=df1['selftext'].str.lower()
df1['selftext']

0       we are hoping to improve services for young ad...
1                                      whats goooood hoes
2                         and if there is gimme an invite
3       hello! i'm an msc student at the ucl institute...
4       i wasn't sure which subreddit to post this too...
                              ...                        
4208    if you feel up to answering, what are your pla...
4209    do not ask me what we had to do to acquire the...
4210    hi there! \n\nmy name is rose. i’m 22 years ol...
4211    22 m from australia we are in a strict lockdow...
4212    i haven't made posts like these in a long time...
Name: selftext, Length: 4213, dtype: object

In [4]:
#remove emoji
def remove_emoji(text):
    text = text.encode('ascii', 'ignore').decode()
    return text

In [5]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_emoji(x))
df1['selftext'][0]

'we are hoping to improve services for young adults that have been exposed to relationship abuse. if you are 18-25 years old and have experienced abuse in a relationship then please take some time to complete a short online survey at www.therepairproject.wordpress.com. a donation will be made to a relevant charity for each person who takes part in this research. thank you!'

In [6]:
#now we remove url, punctuations, special characters
import re
def remove_other(x):
    x = re.sub("\$"," ", x) #remove $
    x = re.sub("https*\S+", " ", x) #remove url
    x = re.sub("\'\w+", '', x) #remove i'm,we're,let's after the '
    x = re.sub("[0-9]+", '', x) #remove numbers
    return x

In [7]:
df1['selftext']=df1['selftext'].apply(lambda x: remove_other(x))
df1

,index,selftext,num_comments,score,title
0,0,we are hoping to improve services for young ad...,0,1,Have you experienced abuse in a relationship?
1,5,whats goooood hoes,3,6,SUP DISCORD BITCHES
2,7,and if there is gimme an invite,1,1,Should be a discord for this subreddit
3,8,hello! i an msc student at the ucl institute o...,0,1,Looking for participants for a study on bullying!
4,9,"i wasn sure which subreddit to post this too, ...",0,2,Is My Dad Doing This Out of Spite?
...,...,...,...,...,...
4208,6023,"if you feel up to answering, what are your pla...",14,1,Hello humanoes! I hope y'all have a good day t...
4209,6025,do not ask me what we had to do to acquire the...,0,1,And Now we have a 20 foot tall inflatable gori...
4210,6027,"hi there! \n\nmy name is rose. im years old, ...",1,1,Advice about a Job/Life
4211,6028,m from australia we are in a strict lockdown ...,1,1,Anyone up for a conversation?


Now we use TextBlob for sentiment analysis 

In [8]:
from textblob import TextBlob

In [9]:
pol = []
sub = []

for text in df1['selftext']:
    selftext = TextBlob(text)
    pol.append(selftext.sentiment.polarity)
    sub.append(selftext.sentiment.subjectivity)
    
    
# assign data of lists.  
data = {'Title':df1['title'],'Text': df1['selftext'], 'polarity': pol, 'subjectivity':sub}  
  
# Create DataFrame  
df2 = pd.DataFrame(data)  
df2

,Title,Text,polarity,subjectivity
0,Have you experienced abuse in a relationship?,we are hoping to improve services for young ad...,0.266667,0.516667
1,SUP DISCORD BITCHES,whats goooood hoes,0.000000,0.000000
2,Should be a discord for this subreddit,and if there is gimme an invite,0.000000,0.000000
3,Looking for participants for a study on bullying!,hello! i an msc student at the ucl institute o...,0.221212,0.281061
4,Is My Dad Doing This Out of Spite?,"i wasn sure which subreddit to post this too, ...",0.055517,0.386140
...,...,...,...,...
4208,Hello humanoes! I hope y'all have a good day t...,"if you feel up to answering, what are your pla...",0.000000,0.000000
4209,And Now we have a 20 foot tall inflatable gori...,do not ask me what we had to do to acquire the...,0.250000,0.333333
4210,Advice about a Job/Life,"hi there! \n\nmy name is rose. im years old, ...",0.261453,0.708333
4211,Anyone up for a conversation?,m from australia we are in a strict lockdown ...,0.200000,0.300000


In [10]:
df2['pol_estimate']=0
df2['pol_estimate'].loc[df2['polarity']>0.2]=1
df2['pol_estimate'].loc[df2['polarity']< -0.2]=-1
df2['sub_estimate']=0
df2['sub_estimate'].loc[df2['subjectivity']>0.6]=1
df2['sub_estimate'].loc[df2['subjectivity']<0.4]=-1
df2

C:\Users\jim13\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\jim13\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
C:\Users\jim13\anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

,Title,Text,polarity,subjectivity,pol_estimate,sub_estimate
0,Have you experienced abuse in a relationship?,we are hoping to improve services for young ad...,0.266667,0.516667,1,0
1,SUP DISCORD BITCHES,whats goooood hoes,0.000000,0.000000,0,-1
2,Should be a discord for this subreddit,and if there is gimme an invite,0.000000,0.000000,0,-1
3,Looking for participants for a study on bullying!,hello! i an msc student at the ucl institute o...,0.221212,0.281061,1,-1
4,Is My Dad Doing This Out of Spite?,"i wasn sure which subreddit to post this too, ...",0.055517,0.386140,0,-1
...,...,...,...,...,...,...
4208,Hello humanoes! I hope y'all have a good day t...,"if you feel up to answering, what are your pla...",0.000000,0.000000,0,-1
4209,And Now we have a 20 foot tall inflatable gori...,do not ask me what we had to do to acquire the...,0.250000,0.333333,1,-1
4210,Advice about a Job/Life,"hi there! \n\nmy name is rose. im years old, ...",0.261453,0.708333,1,1
4211,Anyone up for a conversation?,m from australia we are in a strict lockdown ...,0.200000,0.300000,0,-1


In [11]:
pol_mean = df2['polarity'].mean()
pol_mean

0.07336714541441464

In [12]:
sub_mean = df2['subjectivity'].mean()
sub_mean

0.44276407540119217

In [13]:
overall_pol = pol_mean = df2['pol_estimate'].mean()
overall_pol

0.14953714692618086

In [14]:
overall_sub = pol_mean = df2['sub_estimate'].mean()
overall_sub

-0.10704960835509138

In [20]:
index=df2[df2['Text'].str.contains("great")].index
#example of calculating average sentiment score of a given word in any posts
df2.iloc[index]['polarity'].mean()

0.18613003816167006

In [21]:
index=df2[df2['Text'].str.contains("great")].index
#example of calculating average sentiment score of a given word in positive posts
df2.iloc[index][df2['pol_estimate']==1]['polarity'].mean()

<ipython-input-21-e7ca6f9e5f30>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2.iloc[index][df2['pol_estimate']==1]['polarity'].mean()


0.36293051805142873

In [22]:
index=df2[df2['Text'].str.contains("bad")].index
#example of calculating average sentiment score of a given word in negative posts
df2.iloc[index][df2['pol_estimate']==-1]['polarity'].mean()

<ipython-input-22-6f3d9e536141>:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2.iloc[index][df2['pol_estimate']==-1]['polarity'].mean()


-0.30955211565984964